# 抖音用户数据分析
    1.特征指标构建

## 0.项目介绍与说明

### 数据集说明
|字段名|释义|字段名|释义|字段名|释义|
|-----|---|-----|---|------|---|
|uid|用户id|user_city|用户城市|item_id|作品id|
|author_id|作者id|item_city|作者城市|channel|作品频道|
|finish|是否看完|like|是否点赞|music_id|音乐id|
|duration_time|作品时长|real_time|具体发布时间|H、data|时、天(发布)|


In [1]:
import pandas as pd
import numpy as np

## 1.数据简单处理

In [2]:
df = pd.read_csv(r'D:\计算机编程\Analysis Dataset\douyin_dataset\douyin_dataset.csv')
df.head()

,Unnamed: 0,uid,user_city,item_id,author_id,item_city,channel,finish,like,music_id,duration_time,real_time,H,date
0,3,15692,109.0,691661,18212,213.0,0,0,0,11513.0,10,2019-10-28 21:55:10,21,2019-10-28
1,5,44071,80.0,1243212,34500,68.0,0,0,0,1274.0,9,2019-10-21 22:27:03,22,2019-10-21
2,16,10902,202.0,3845855,634066,113.0,0,0,0,762.0,10,2019-10-26 00:38:51,0,2019-10-26
3,19,25300,21.0,3929579,214923,330.0,0,0,0,2332.0,15,2019-10-25 20:36:25,20,2019-10-25
4,24,3656,138.0,2572269,182680,80.0,0,0,0,238.0,9,2019-10-21 20:46:29,20,2019-10-21


删除无效字段

In [3]:
del df['Unnamed: 0']

In [4]:
# 基本数据信息
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1737312 entries, 0 to 1737311
Data columns (total 13 columns):
 #   Column         Dtype  
---  ------         -----  
 0   uid            int64  
 1   user_city      float64
 2   item_id        int64  
 3   author_id      int64  
 4   item_city      float64
 5   channel        int64  
 6   finish         int64  
 7   like           int64  
 8   music_id       float64
 9   duration_time  int64  
 10  real_time      object 
 11  H              int64  
 12  date           object 
dtypes: float64(3), int64(8), object(2)
memory usage: 172.3+ MB


## 2.特征指标构建

### 用户指标分析
* 站在用户的角度，涉及到浏览量，点赞量，浏览的作品、作者、BGM的总数等
> 浏览量
> 点赞量
> 完整观看数
> 观看作品数
> 观看作者数
> 去过城市数
> 观看作品城市数
> 观看配乐数
> 观看作品平均时长
### 作者指标分析
* 站在作者的角度，涉及到总浏览量，总点赞量等
> 总浏览量
> 总点击量
> 总关完量
> 总作品数
> 作品平均时长
> 使用配乐数量
> 日发布作品数
### 作品指标分析
> 浏览量
> 点赞量
> 发布城市

## 3.特征指标统计分析

### 3.1用户特征指标统计分析

In [5]:
user_df = pd.DataFrame()
user_df['uid'] = df.groupby('uid')['like'].count().index.tolist() # 将所有用户的uid提取为uid列
user_df.set_index('uid',inplace=True) # 设置uid为索引
user_df['浏览量'] = df.groupby('uid')['like'].count() # 统计对应uid下的浏览量
user_df['点赞量'] = df.groupby('uid')['like'].sum() # 统计uid下的点赞量
user_df['观看作者数'] = df.groupby(['uid']).agg({'author_id':pd.Series.nunique}) # 观看作者数
user_df['观看作品数'] = df.groupby(['uid']).agg({'item_id':pd.Series.nunique}) # 观看作品数
user_df['观看作品平均时长'] = df.groupby(['uid'])['duration_time'].mean() # 观看作品平均时长
user_df['观看配乐数'] = df.groupby(['uid']).agg({'music_id':pd.Series.nunique})
user_df['观看完整数'] = df.groupby(['uid'])['finish'].sum() # 统计uid下的完整观看数
# 统计对应uid用户去过的城市数量
user_df['去过的城市数'] = df.groupby(['uid']).agg({'user_city':pd.Series.nunique})
# 统计对应uid用户看的作品所在的城市数量
user_df['观看作品城市数'] = df.groupby(['uid']).agg({'item_city':pd.Series.nunique})


user_df.to_csv('用户特征.csv',encoding='utf-8-sig')

### 3.2作者特征统计分析

In [10]:
author_df = pd.DataFrame()
author_df['author_id'] = df.groupby('author_id')['like'].count().index.tolist()
author_df.set_index('author_id',inplace=True)
author_df['总浏览量'] = df.groupby('author_id')['like'].count()
author_df['总点赞量'] = df.groupby('author_id')['like'].sum()
author_df['总观完量'] = df.groupby('author_id')['finish'].sum()
author_df['总作品数'] = df.groupby('author_id').agg({'item_id':pd.Series.nunique})

item_time = df.groupby(['author_id','item_id']).mean(numeric_only=True).reset_index()
author_df['作品平均时间'] = item_time.groupby('author_id')['duration_time'].mean()

author_df['使用配乐数量'] = df.groupby('author_id').agg({'music_id':pd.Series.nunique})
author_df['发布作品日数'] = df.groupby('author_id').agg({'real_time':pd.Series.nunique})

author_days = df.groupby('author_id')['date']
_ = pd.to_datetime(author_days.max()) - pd.to_datetime(author_days.min())
author_df['创作活跃度(日)'] = _.map(lambda x: x/np.timedelta64(1,'D'))
author_df['去过的城市数'] = df.groupby(['author_id']).agg({'item_city':pd.Series.nunique})
author_df.describe()

author_df.to_csv('作者特征.csv',encoding='utf_8_sig')

### 3.3作品特征统计分析

In [19]:
item_df = pd.DataFrame()
item_df['item_id'] = df.groupby('item_id')['like'].count().index.tolist()
item_df.set_index('item_id',inplace=True)
item_df['浏览量'] = df.groupby('item_id')['like'].count()
item_df['点赞量'] = df.groupby('item_id')['like'].sum()
item_df['发布城市'] = df.groupby('item_id')['item_city'].mean()  # 源数据中的item_city字段是数字类型
item_df['背景音乐'] = df.groupby('item_id')['music_id'].mean()

In [21]:
item_df.to_csv('作品特征.csv',encoding='utf_8_sig')

## 4.总结
通过对浏览行为数据的认识和分析，构建并提取了用户、作者、作品的特征，特征的构造和提取并不是唯一，如可以统计用户点赞率、作者的观众数等